# Quinta lezione

I notebook possono essere usati per pubblicazioni o per ausilo a pubblicazioni (cioè come integrazione a pubblicazioni in formato tradizionale).

Proviamo a produrre un ideale rapporto sull'analisi di un set di misure.

*  Conterrà spegazione del contesto teorico (qui sotto un esempio).

*  Conterrà i dati e la procedura usata per elaborarli.

È realistico immaginare che il rapporto venga pubblicato in formato `.html`. Questo permette di includere grafica interattiva ma NON offre la possibilità di interagire dinamicamente con il documento (modificando il codice o dei parametri). Mettere a disposizione il sorgente in formato `.ipynb` è auspicabile. Ma potrebbe non essere sufficiente per chi vuole fruire immediatamente del codice (salvarlo in un ambiente dove si ha accesso ad un installazione di Jupyter potrebbe essere "a click too far" per qualche lettore).

Esistono un paio di possibilità.<br><br>

*   [Binder](https://mybinder.org/) è un servizio offerto da Github per interagire con notebooks. Permette di creare un  [link](https://mybinder.org/v2/gh/domenicozambella/BioTeIndu19/master?filepath=lezioni%2F5_prelezione.ipynb) da condividere. Spesso nei documenti il link viene incluso con il suo caratteristico bottone 
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/domenicozambella/BioTeIndu19/master?filepath=lezioni%2F5_prelezione.ipynb) 
Ha due princpali difetti. Lento a caricare (perché crea una macchina virtuale) e si disattiva dopo un breve periodo di inattività.<br><br>

*   [Google Coloboratory](https://colab.research.google.com/notebooks/welcome.ipynb) (anche detto Colab) è un servizio analogo offerto da Google. 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/domenicozambella/BioTeIndu19/blob/master/lezioni/5_prelezione.ipynb)
Ha comme princiali difetti l'interfaccia non standard e il fatto che funziona solo con Python notebooks (niente R, Julia, ecc.).

<br><br>

Esempio di introduzione teorica:

<hr><br>

# Theoretical Backgound: the Michaelis-Menten equation


Mathematical models of enzymes can take many forms, but the best known is the [Michaelis-Menten equation](https://en.wikipedia.org/wiki/Michaelis–Menten_kinetics) which considers the the mechanism of an irreversible enzyme ($E$) producing product ($P$) from substrate ($S$):

$$
S + E 
\overset{k_f}{\underset{k_r}{\rightleftharpoons}} 
SE 
\overset{k_\text{cat}}{\longrightarrow} 
S + P 
\ \implies\ 
v = \frac{\mathrm{d}p}{\mathrm{d}t} = \frac{V_\text{max}\cdot s}{K_m + s}
$$

- $s$ is the *concentration of substrate $S$*
- $p$ is the *concentration of product $P$*
- $k_f$ is called the *association constant*
- $k_r$ is called the *disassociation constant*
- $k_\text{cat}$ is called the *turnover number*
- $K_m = (k_\text{cat} + k_{r})\;/\;k_f$ is called the *Michaelis constant*
- $e_0$ is the *initial concentration of enzyme*
- $V_\text{max} = k_\text{cat}e_0$ is the *maximum catalytic rate*

<br><hr>

I dati li leggiamo direttamente dalla repositoria su github, perché rende il notebook più facilmente esportabile.

In [1]:
import pandas as pd
URL = 'https://raw.githubusercontent.com/domenicozambella/BioTeIndu19/master/dati/mm1.csv'
df = pd.read_csv(URL)

Tabelle così piccole possono anche essere inserite manualmente con la seguente sintassi.

In [2]:
data = dict(s = [0.5,1,2.5,3.5,5,7.5,10,15,25,50,70,75,100],
            v = [0.6,1.1,2.1,2.3,3.7,3.,4.3,4.8,5.3,6.0,5.1,5.7,5.8],
           )
df = pd.DataFrame(data)

Per prima cosa importiamo importiamo il pacchetto grafico. Salviamo in una variabile, `param`, alcuni parametri grafici che useremo in tutte le figure sottostanti (si ignori per il momento la sintassi del comando).

In [3]:
from bokeh.plotting import figure, show, output_notebook
param = dict(width = 700, height = 250,
             tools = 'wheel_zoom, reset,pan, box_zoom',
             tooltips = [( 'substrate',   '@s'), ( 'velocity',   '@v')],
            )

In [4]:
p = figure(x_axis_label='Concentrazione substrato',
           y_axis_label='Velocità di reazione',
           **param,
           )
p.circle( 's', 'v', source=df, size=5 )
output_notebook()
show( p )

Loading BokehJS ...

## Linearizzazione 1: Lineweaver-Burk

$$
\dfrac{1}{v}
=
\dfrac{K_\textrm{m}}{V_\textrm{max}}\cdot\dfrac{1}{s}\ +\ \dfrac{1}{V_\textrm{max}}
$$


Espandiamo il dataframe con i reciproci delle prime colonne

In [5]:
df['1/s'] = df['s'].map(lambda x: 1/x)
df['1/v'] = df['v'].map(lambda x: 1/x)

Ora plottiamo `1/v` su `1/s`.

In [6]:
p2 = figure(title = 'Linearizzazione di Lineweaver-Burk',
            x_axis_label=' 1/s',
            y_axis_label='1/v',
            **param,
            )
p2.circle( '1/s', '1/v', source=df, size=5 )
output_notebook()
show( p2 )

Loading BokehJS ...

In [7]:
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress( df['1/s'], df['1/v'] )
slope, intercept, r_value, p_value, std_err

(0.7480681984452995,
 0.17125207846346693,
 0.9976695813464803,
 3.5156919427820225e-14,
 0.01542541728718807)

In [8]:
from numpy import linspace
x = linspace( 0,2,100  )
y = x*slope + intercept
p2.line( x, y)
output_notebook()
show( p2 )
'Vmax = {}, Km = {}'.format(1/intercept, slope/intercept) 

Loading BokehJS ...

'Vmax = 5.839345186186042, Km = 4.368228433530424'

## Linearizzazione 2: Eadie-Hofstee

$$
v
=
-K_\textrm{m}\dfrac{v}{s}\ +\ V_\textrm{max}
$$

In [9]:
df['v/s'] = df[ ['s','v'] ].apply(lambda x: x[1]/x[0], axis=1)
df

,s,v,1/s,1/v,v/s
0,0.5,0.6,2.000000,1.666667,1.200000
1,1.0,1.1,1.000000,0.909091,1.100000
2,2.5,2.1,0.400000,0.476190,0.840000
3,3.5,2.3,0.285714,0.434783,0.657143
4,5.0,3.7,0.200000,0.270270,0.740000
5,7.5,3.0,0.133333,0.333333,0.400000
6,10.0,4.3,0.100000,0.232558,0.430000
7,15.0,4.8,0.066667,0.208333,0.320000
8,25.0,5.3,0.040000,0.188679,0.212000
9,50.0,6.0,0.020000,0.166667,0.120000


In [10]:
p3 = figure(title = 'Eadie-Hofstee',
            x_axis_label='v/s',
            y_axis_label='v',
            **param,
          )
p3.circle( 'v/s', 'v', source=df, size=5 )
output_notebook()
show( p3 )

Loading BokehJS ...

Interpoliamo questi valori con una regressione lineare.

In [11]:
slope, intercept, r_value, p_value, std_err = stats.linregress( df['v/s'], df['v'] )
x = linspace( 0,1.2,100  )
y = x*slope + intercept
p3.line( x, y)
output_notebook()
show( p3 )
'Vmax = {}, Km = {}'.format(intercept, -slope) 

Loading BokehJS ...

'Vmax = 5.961030175109625, Km = 4.448023173213158'

In [12]:
# This cell loads some html style files (it may be either run/ignored/deleted)
from IPython.core.display import HTML
with open( '../lezioni/style/custom.css', 'r' ) as f: html_style = f.read()
HTML( html_style )